In [ ]:
!pip install fastfeedforward

In [ ]:
import sys
sys.path.insert(0, "../fastfeedforward")

from tqdm import tqdm

import torch
import torch.nn as nn
from fastfeedforward import FFF
import matplotlib.pyplot as plt

print(dir(FFF))
import math

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_call_impl', '_compiled_call_impl', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_load_from_state_dict', '_maybe_warn_non_full_backward_hook', '_named_members', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_version', '_wrapped_call_impl', 'add_module', 'apply', 'bfloat16', 'buffers', 'call_super_init', 'children', 'compile', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'eval_forward', 'extra_repr', 'float', 'forward', 'get_buff

In [ ]:
n_epochs = 5
activation = nn.ReLU()
batch_size = 256

leaf_width = 2          # [1, 2, 4, 8]
training_width = 32     # [16, 32, 64, 128]    # number of all neurons in leafs

depth = int(math.log2(training_width/leaf_width))
print(depth)
"""
depth = []
for w in leaf_width:
    for ℓ in training_width:
        d = math.log2(w / ℓ)
        depth.append(d)
"""
entropy_effect = 3.0

leaf_dropout = 0.0
region_leak = 0.0

4


In [ ]:
# Load the datasets
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# MNIST dataset
dataset_mnist = datasets.MNIST('data', download=True, train=True, transform=transform)
dataloader_mnist = torch.utils.data.DataLoader(dataset_mnist, batch_size=batch_size, shuffle=True)

# MNIST testing dataset
dataset_mnist_test = datasets.MNIST('data', download=True, train=False, transform=transform)
dataloader_mnist_test = torch.utils.data.DataLoader(dataset_mnist, batch_size=batch_size, shuffle=True)

# USPS dataset
dataset_usps = datasets.USPS('data_usps', download=True, train=True, transform=transform)
dataloader_usps = torch.utils.data.DataLoader(dataset_usps, batch_size=batch_size, shuffle=True)

# USPS testing dataset
dataset_usps_test = datasets.USPS('data_usps', download=True, train=False, transform=transform)
dataloader_usps_test = torch.utils.data.DataLoader(dataset_usps_test, batch_size=batch_size, shuffle=True)

# FashionMNIST dataset
dataset_fashion_mnist = datasets.FashionMNIST('data_fashion_mnist', download=True, train=True, transform=transform)
dataloader_fashion_mnist = torch.utils.data.DataLoader(dataset_fashion_mnist, batch_size=batch_size, shuffle=True)

# FashionMNIST testing dataset
dataset_fashion_mnist_test = datasets.FashionMNIST('data_fashion_mnist', download=True, train=False, transform=transform)
dataloader_fashion_mnist_test = torch.utils.data.DataLoader(dataset_fashion_mnist_test, batch_size=batch_size, shuffle=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 109775782.00it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 29755758.74it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30782442.68it/s]


...

Extracting data_fashion_mnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data_fashion_mnist/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4954597.59it/s]


Extracting data_fashion_mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data_fashion_mnist/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12324359.01it/s]

Extracting data_fashion_mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data_fashion_mnist/FashionMNIST/raw



In [ ]:
import math
from typing import List, Optional

import torch
from torch import Tensor

class LocalAdam(torch.optim.Optimizer):
    """Implements Adam algorithm with local batch size support.

    All input parameters are as per :class:`torch.optim.Adam` except for the following:
     - `usage`: a tensor of shape (N,) where N is the number of parameter blocks (to have separate gradient accumulations) in the group. If specified, the optimizer will only update parameters where usage[i] >= local_batch_size[i].
     - `local_batch_size`: a tensor of shape (N,) where N is the number of parameter blocks (to have separate gradient accumulations) in the group. If specified, the optimizer will only update parameters where usage[i] >= local_batch_size[i].
     - `differentiable`, `fusion`, and `foreach` from the original implementation are not supported.
    """
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False, *,
                 usage: Optional[torch.Tensor] = None, local_batch_size: Optional[int] = None,
                 maximize: bool = False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if usage is not None and not isinstance(usage, torch.Tensor):
            raise ValueError("Parameter usage must be a tensor")
        if local_batch_size is not None and local_batch_size.any() <= 0:
            raise ValueError("Invalid local_batch_size value: {}, must be > 0 or unspecified altogether (None)".format(local_batch_size))

        params = list(params)

        for param in params:
            if not isinstance(param, dict):
                continue

            if 'usage' in param:
                if not isinstance(param['usage'], torch.Tensor):
                    raise ValueError("Parameter usage must be a tensor")
                for p in param['params']:
                    if p.size(0) != param['usage'].size(0):
                        raise ValueError("Every tensor in param['params'] must have the same size(0) as param['usage'].size(0)")

            if 'local_batch_size' in param:
                if not isinstance(param['local_batch_size'], int):
                    raise ValueError("Parameter local_batch_size must be an int")
                if param['local_batch_size'] <= 0:
                    raise ValueError("Parameter local_batch_size must be >0")

        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad,
                        maximize=maximize,
                        usage=usage, local_batch_size=local_batch_size,)
        super().__init__(params, defaults)

    def __setstate__(self, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)
            group.setdefault('maximize', False)
            group.setdefault('usage', None)
            group.setdefault('local_batch_size', None)

        state_values = list(self.state.values())
        step_is_tensor = (len(state_values) != 0) and torch.is_tensor(state_values[0]['step'])
        if not step_is_tensor:
            for s in state_values:
                s['step'] = torch.tensor(s['step'], dtype=torch.float32)

    def _init_group(
        self,
        group,
        params_with_grad,
        grads,
        u_list,
        lbs_list,
        exp_avgs,
        exp_avg_sqs,
        max_exp_avg_sqs,
        state_steps
    ):
        for p in group['params']:
            if p.grad is not None:
                params_with_grad.append(p)
                if p.grad.is_sparse:
                    raise RuntimeError('LocalAdam does not support sparse gradients; perhaps consider torch.optim.SparseAdam')
                grads.append(p.grad)

                if group['usage'] is not None and group['local_batch_size'] is not None:
                    u_list.append(group['usage'])
                    lbs_list.append(group['local_batch_size'])
                else:
                    u_list.append(None)
                    lbs_list.append(None)

                state = self.state[p]
                # Lazy state initialization
                if len(state) == 0:
                    descent_controlled_by_usage = group['usage'] is not None and group['local_batch_size'] is not None
                    state['step'] = (
                        torch.tensor(0.) if not descent_controlled_by_usage else torch.zeros_like(group['usage'], dtype=torch.float32, memory_format=torch.preserve_format)
                    )
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    if group['amsgrad']:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                exp_avgs.append(state['exp_avg'])
                exp_avg_sqs.append(state['exp_avg_sq'])

                if group['amsgrad']:
                    max_exp_avg_sqs.append(state['max_exp_avg_sq'])
                state_steps.append(state['step'])

    def zero_grad(self, set_to_none: bool = False):
        usage_zeroing_per_usage_holder = {}
        for group in self.param_groups:
            u = group['usage']
            if u is not None and group['local_batch_size'] is not None:
                where_to_zero = u >= group['local_batch_size']
            else:
                where_to_zero = None

            for p in group['params']:
                if p.grad is not None:
                    if where_to_zero is None:
                        if set_to_none:
                            p.grad = None
                        else:
                            if p.grad.grad_fn is not None:
                                p.grad.detach_()
                            # DO NOT MODIFY requires_grad even though the base implementation does; ever!
                            p.grad.zero_()
                    else:
                        p.grad[where_to_zero] = 0

            if where_to_zero is not None:
                if u in usage_zeroing_per_usage_holder:
                    usage_zeroing_per_usage_holder[u].logical_or_(where_to_zero)
                else:
                    usage_zeroing_per_usage_holder[u] = where_to_zero.clone().detach()

        # now we can zero out the gradients for each usage holder
        for u, where_to_zero in usage_zeroing_per_usage_holder.items():
            u[where_to_zero] = 0

    def step(self, closure=None):
        """Performs a single optimization step.

        Args:
            closure (Callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            params_with_grad = []
            grads = []
            u_list = []
            lbs_list = []
            exp_avgs = []
            exp_avg_sqs = []
            max_exp_avg_sqs = []
            state_steps = []
            beta1, beta2 = group['betas']

            self._init_group(
                group,
                params_with_grad,
                grads,
                u_list,
                lbs_list,
                exp_avgs,
                exp_avg_sqs,
                max_exp_avg_sqs,
                state_steps)

            adam(
                params_with_grad,
                grads,
                u_list,
                lbs_list,
                exp_avgs,
                exp_avg_sqs,
                max_exp_avg_sqs,
                state_steps,
                amsgrad=group['amsgrad'],
                beta1=beta1,
                beta2=beta2,
                lr=group['lr'],
                weight_decay=group['weight_decay'],
                eps=group['eps'],
                maximize=group['maximize'],
                grad_scale=getattr(self, "grad_scale", None),
                found_inf=getattr(self, "found_inf", None),
            )

        return loss

def adam(params: List[Tensor],
         grads: List[Tensor],
         u_list: List[Optional[Tensor]],
         lbs_list: List[Optional[Tensor]],
         exp_avgs: List[Tensor],
         exp_avg_sqs: List[Tensor],
         max_exp_avg_sqs: List[Tensor],
         state_steps: List[Tensor],
         # kwonly args with defaults are not supported by functions compiled with torchscript issue #70627
         # setting this as kwarg for now as functional API is compiled by torch/distributed/optim
         grad_scale: Optional[Tensor] = None,
         found_inf: Optional[Tensor] = None,
         *,
         amsgrad: bool,
         beta1: float,
         beta2: float,
         lr: float,
         weight_decay: float,
         eps: float,
         maximize: bool):
    r"""Functional API that performs Adam algorithm computation.
    See :class:`~torch.optim.Adam` for details.
    """

    if not all(isinstance(t, torch.Tensor) for t in state_steps):
        raise RuntimeError("API has changed, `state_steps` argument must contain a list of singleton tensors")

    func = _single_tensor_adam

    func(params,
         grads,
         u_list,
         lbs_list,
         exp_avgs,
         exp_avg_sqs,
         max_exp_avg_sqs,
         state_steps,
         amsgrad=amsgrad,
         beta1=beta1,
         beta2=beta2,
         lr=lr,
         weight_decay=weight_decay,
         eps=eps,
         maximize=maximize,
         grad_scale=grad_scale,
         found_inf=found_inf)

@torch.no_grad()
def _single_tensor_adam(params: List[Tensor],
                        grads: List[Tensor],
                        u_list: List[Optional[Tensor]],
                        lbs_list: List[Optional[Tensor]],
                        exp_avgs: List[Tensor],
                        exp_avg_sqs: List[Tensor],
                        max_exp_avg_sqs: List[Tensor],
                        state_steps: List[Tensor],
                        grad_scale: Optional[Tensor],
                        found_inf: Optional[Tensor],
                        *,
                        amsgrad: bool,
                        beta1: float,
                        beta2: float,
                        lr: float,
                        weight_decay: float,
                        eps: float,
                        maximize: bool
                        ):

    assert grad_scale is None and found_inf is None

    for i, param in enumerate(params):
        grad = grads[i] if not maximize else -grads[i]
        exp_avg = exp_avgs[i]
        exp_avg_sq = exp_avg_sqs[i]
        u = u_list[i]
        lbs = lbs_list[i]

        # find out where updates are due
        if u is not None and lbs is not None:
            to_update = u >= lbs
            to_update_expanded = to_update.reshape(to_update.shape + (1,) * (len(grad.shape) - len(to_update.shape)))
            to_update_expanded = to_update_expanded.expand_as(grad)
        else:
            to_update = None

        # correct the grad where more accumulations have happened
        if to_update is not None:
            correction = torch.where(to_update, u / lbs, torch.ones_like(u))
            correction_expanded = correction.reshape(correction.shape + (1,) * (len(grad.shape) - len(correction.shape)))
            correction_expanded = correction_expanded.expand_as(grad)
            grad = grad / correction_expanded

        # update the step counter where an update is imminent
        if to_update is not None:
            state_steps[i].copy_(torch.where(to_update, state_steps[i] + 1, state_steps[i]))
        else:
            state_steps[i].add_(1)

        if weight_decay != 0:
            grad = grad.add(param, alpha=weight_decay)

        if torch.is_complex(param):
            grad = torch.view_as_real(grad)
            exp_avg = torch.view_as_real(exp_avg)
            exp_avg_sq = torch.view_as_real(exp_avg_sq)
            param = torch.view_as_real(param)

        # Decay the first and second moment running average coefficient
        if to_update is not None:
            exp_avg.copy_(torch.where(to_update_expanded, exp_avg.mul_(beta1).add(grad, alpha=1 - beta1), exp_avg))
            exp_avg_sq.copy_(torch.where(to_update_expanded, exp_avg_sq.mul(beta2).addcmul(grad, grad.conj(), value=1 - beta2), exp_avg_sq))
        else:
            exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
            exp_avg_sq.mul_(beta2).addcmul_(grad, grad.conj(), value=1 - beta2)

        step = state_steps[i]

        bias_correction1 = torch.where(to_update, 1 - beta1 ** step, torch.ones_like(step))
        bias_correction2 = torch.where(to_update, 1 - beta2 ** step, torch.ones_like(step))
        bias_correction2 = bias_correction2.reshape(bias_correction2.shape + (1,) * (len(exp_avg_sq.shape) - len(bias_correction2.shape)))

        step_size = lr / bias_correction1
        step_size = step_size.reshape(step_size.shape + (1,) * (len(exp_avg.shape) - len(step_size.shape)))

        bias_correction2_sqrt = torch.sqrt(bias_correction2)

        if amsgrad:
            # Maintains the maximum of all 2nd moment running avg. till now
            if to_update is not None:
                torch.maximum(
                    max_exp_avg_sqs[i],
                    torch.where(to_update_expanded, exp_avg_sq, max_exp_avg_sqs[i]),
                    out=max_exp_avg_sqs[i]
                )
            else:
                torch.maximum(max_exp_avg_sqs[i], exp_avg_sq, out=max_exp_avg_sqs[i])

            # Use the max. for normalizing running avg. of gradient
            denom = (max_exp_avg_sqs[i].sqrt() / bias_correction2_sqrt).add_(eps)
        else:
            denom = (exp_avg_sq.sqrt() / bias_correction2_sqrt).add_(eps)

        if to_update is None:
            param.copy_(param + exp_avg / denom * -step_size)
        else:
            param.copy_(torch.where(
                to_update_expanded,
                param + exp_avg / denom * -step_size,
                param
            ))

In [ ]:
# setup the FFF model
model = FFF(input_width=784, leaf_width=leaf_width, output_width=10, depth=depth, activation=activation, dropout=leaf_dropout, region_leak=region_leak)

In [ ]:
# train the model
training_losses = []
training_accuracies = []
training_entropies = []
epoch_testing_losses = []
epoch_testing_accuracies = []

print(model.parameters())

parameters = list(model.parameters())

for param_group in parameters:
    print(param_group)



num_blocks = 16 #len(parameters)
usage = torch.tensor([0] * num_blocks)
print(usage)
local_batch_size = torch.tensor([32] * num_blocks)
print(local_batch_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()
for epoch in range(n_epochs):
	print("Epoch", epoch)

	model.train()
	for batch_images, batch_labels in tqdm(dataloader_mnist):
		optimizer.zero_grad()

		output, node_entropies = model(batch_images.view(-1, 784), return_entropies=True)
		node_entropy_mean = node_entropies.mean()
		loss = criterion(output, batch_labels) + entropy_effect * node_entropy_mean
		accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

		training_losses.append(loss.item())
		training_accuracies.append(accuracy.item())
		training_entropies.append(node_entropy_mean.item())

		loss.backward()
		optimizer.step()

	# test the model
	model.eval()
	testing_losses = []
	testing_accuracies = []
	for batch_images, batch_labels in tqdm(dataloader_mnist_test):
		output = model(batch_images.view(-1, 784))
		loss = criterion(output, batch_labels)
		accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

		testing_losses.append(loss.item())
		testing_accuracies.append(accuracy.item())
	epoch_testing_losses.append(sum(testing_losses) / len(testing_losses))
	epoch_testing_accuracies.append(sum(testing_accuracies) / len(testing_accuracies))

In [ ]:
def train1(model, dataloader_training, dataloader_testing, n_epochs=10, entropy_effect=3.0, learning_rate=0.02, patience=3):

    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    best_training_loss = float('inf')
    consecutive_no_improvement = 0

    # Training loop
    for epoch in range(n_epochs):
        print("Epoch", epoch)
        training_losses = []
        training_accuracies = []

        # Training
        model.train()
        for batch_images, batch_labels in tqdm(dataloader_training):

            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            optimizer.zero_grad()

            output, node_entropies = model(batch_images.view(-1, 784), return_entropies=True)
            node_entropy_mean = node_entropies.mean()
            loss = criterion(output, batch_labels) + entropy_effect * node_entropy_mean
            accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

            training_losses.append(loss.item())
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        # Calculate training metrics
        training_loss = sum(training_losses)
        print(f"Epoch {epoch} - Training Loss: {training_loss}")

        training_accuracy = sum(training_accuracies) / len(training_accuracies)
        print(f"Epoch {epoch} - Training Accuracy: {training_accuracy}")

        """
        # Testing
        model.eval()
        testing_losses = []
        testing_accuracies = []
        for batch_images, batch_labels in tqdm(dataloader_testing):

            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            output = model(batch_images.view(-1, 784))
            loss = criterion(output, batch_labels)
            accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

            testing_losses.append(loss.item())
            testing_accuracies.append(accuracy.item())

        # Calculate testing metrics
        testing_loss = sum(testing_losses) / len(testing_losses)
        print(f"Epoch {epoch} - Testing Loss: {testing_loss}")

        testing_accuracy = sum(testing_accuracies) / len(testing_accuracies)
        print(f"Epoch {epoch} - Testing Accuracy: {testing_accuracy}")
        """
        # Early Stopping
        if training_loss < best_training_loss:
            best_training_loss = training_loss
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement >= patience:
            print(f"Early stopping.")
            break

    return model

def eval1(model, dataloader_testing):

    criterion = nn.CrossEntropyLoss()

    model.eval()
    # testing_losses = []
    testing_accuracies = []
    for batch_images, batch_labels in tqdm(dataloader_testing):
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
        output = model(batch_images.view(-1, 784))
        # loss = criterion(output, batch_labels)
        accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()
        # testing_losses.append(loss.item())
        testing_accuracies.append(accuracy.item())

    # Calculate testing metrics
    # testing_loss = sum(testing_losses) / len(testing_losses)
    testing_accuracy = sum(testing_accuracies) / len(testing_accuracies)

    return testing_accuracy


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
model = FFF(input_width=784, leaf_width=leaf_width, output_width=10, depth=depth, activation=activation, dropout=leaf_dropout, region_leak=region_leak)
model.to(device)
trained_metrics = train1(model, dataloader_mnist, dataloader_mnist, n_epochs=500, entropy_effect=entropy_effect, learning_rate=0.001, patience=500)
print("\n")
print(eval1(model,dataloader_mnist))

Using device: cuda
Epoch 0


100%|██████████| 235/235 [00:16<00:00, 14.68it/s]


Epoch 0 - Training Loss: 525.8235847949982
Epoch 0 - Training Accuracy: 0.18043550531914893
Epoch 1


 83%|████████▎ | 196/235 [00:14<00:02, 13.60it/s]


KeyboardInterrupt: ignored

In [ ]:
def train2(model, dataloader_training, dataloader_testing, n_epochs=10, entropy_effect=3.0, learning_rate=0.02, patience=3):

    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    best_testing_loss = float('inf')
    consecutive_no_improvement = 0

    # Training loop
    for epoch in range(n_epochs):
        print("Epoch", epoch)
        training_losses = []
        training_accuracies = []

        # Training
        model.train()
        for batch_images, batch_labels in tqdm(dataloader_training):

            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            optimizer.zero_grad()

            output, node_entropies = model(batch_images.view(-1, 784), return_entropies=True)
            node_entropy_mean = node_entropies.mean()
            loss = criterion(output, batch_labels) + entropy_effect * node_entropy_mean
            accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

            training_losses.append(loss.item())
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        # Calculate training metrics
        training_loss = sum(training_losses)
        print(f"Epoch {epoch} - Training Loss: {training_loss}")

        training_accuracy = sum(training_accuracies) / len(training_accuracies)
        print(f"Epoch {epoch} - Training Accuracy: {training_accuracy}")

        # Testing
        model.eval()
        # print(model.training)
        testing_losses = []
        testing_accuracies = []
        for batch_images, batch_labels in tqdm(dataloader_testing):

            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            output = model(batch_images.view(-1, 784))
            loss = criterion(output, batch_labels)
            accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

            testing_losses.append(loss.item())
            testing_accuracies.append(accuracy.item())

        # Calculate testing metrics
        testing_loss = sum(testing_losses)
        print(f"Epoch {epoch} - Testing Loss: {testing_loss}")

        testing_accuracy = sum(testing_accuracies) / len(testing_accuracies)
        print(f"Epoch {epoch} - Testing Accuracy: {testing_accuracy}")

        # Early Stopping
        if testing_loss < best_testing_loss:
            best_testing_loss = testing_loss
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement >= patience:
            print(f"Early stopping.")
            break

    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
model = FFF(input_width=784, leaf_width=16, output_width=10, depth=3, activation=activation, dropout=leaf_dropout, region_leak=region_leak)
model.to(device)
trained_metrics = train2(model, dataloader_mnist, dataloader_mnist_test, n_epochs=500, entropy_effect=3.0, learning_rate=0.001, patience=500)
print("\n")
print(eval1(model,dataloader_mnist_test))
# max acc 0.58 for lr 0.001 batch size 256

Using device: cpu
Epoch 0


100%|██████████| 235/235 [00:19<00:00, 11.80it/s]


Epoch 0 - Training Loss: 220.86019563674927
Epoch 0 - Training Accuracy: 0.7505762412192973
False


100%|██████████| 235/235 [00:26<00:00,  8.83it/s]


Epoch 0 - Testing Loss: 109.03976666927338
Epoch 0 - Testing Accuracy: 0.8688940603682336
Epoch 1


100%|██████████| 235/235 [00:20<00:00, 11.32it/s]


Epoch 1 - Training Loss: 92.03901898860931
Epoch 1 - Training Accuracy: 0.8894337321849579
False


 76%|███████▌  | 179/235 [00:20<00:06,  8.94it/s]


KeyboardInterrupt: 

In [ ]:
# simple ff

class SimpleFeedForward(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFeedForward, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

def train_simple_ff(model, dataloader_training, dataloader_testing, n_epochs=10, learning_rate=0.02, patience=3):

    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    best_testing_loss = float('inf')
    consecutive_no_improvement = 0

    # Training loop
    for epoch in range(n_epochs):
        print("Epoch", epoch)
        training_losses = []
        training_accuracies = []

        # Training
        model.train()
        for batch_images, batch_labels in tqdm(dataloader_training):

            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            optimizer.zero_grad()

            output = model(batch_images.view(-1, 784))
            loss = criterion(output, batch_labels)
            accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

            training_losses.append(loss.item())
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        # Calculate training metrics
        training_loss = sum(training_losses)
        print(f"Epoch {epoch} - Training Loss: {training_loss}")

        training_accuracy = sum(training_accuracies) / len(training_accuracies)
        print(f"Epoch {epoch} - Training Accuracy: {training_accuracy}")

        # Testing
        model.eval()
        testing_losses = []
        testing_accuracies = []
        for batch_images, batch_labels in tqdm(dataloader_testing):

            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            output = model(batch_images.view(-1, 784))
            loss = criterion(output, batch_labels)
            accuracy = (output.argmax(dim=1) == batch_labels).detach().float().mean()

            testing_losses.append(loss.item())
            testing_accuracies.append(accuracy.item())

        # Calculate testing metrics
        testing_loss = sum(testing_losses)
        print(f"Epoch {epoch} - Testing Loss: {testing_loss}")

        testing_accuracy = sum(testing_accuracies) / len(testing_accuracies)
        print(f"Epoch {epoch} - Testing Accuracy: {testing_accuracy}")

        # Early Stopping
        if testing_loss < best_testing_loss:
            best_testing_loss = testing_loss
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement >= patience:
            print(f"Early stopping.")
            break

    return model


In [ ]:
input_size = 784
hidden_size = 128
output_size = 10

# Create a SimpleFeedForward
model = SimpleFeedForward(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

# Move the model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Call the training function
trained_model = train_simple_ff(model, dataloader_mnist, dataloader_mnist_test, n_epochs=500, learning_rate=0.001, patience=500)

# Evaluate the trained model
print("\n")
print(eval1(trained_model, dataloader_mnist_test))


Epoch 0


100%|██████████| 235/235 [00:18<00:00, 12.80it/s]


Epoch 0 - Training Loss: 127.96920724213123
Epoch 0 - Training Accuracy: 0.850376773134191


100%|██████████| 235/235 [00:18<00:00, 12.89it/s]


Epoch 0 - Testing Loss: 77.14136812090874
Epoch 0 - Testing Accuracy: 0.9027648492062346
Epoch 1


 84%|████████▍ | 198/235 [00:15<00:02, 13.00it/s]


KeyboardInterrupt: 

In [ ]:
# Second series of experiements
# datasets SVHN, CIFAR10, CIFAR100
import torch
from torchvision import datasets, transforms

# Set your desired batch size
batch_size = 64

# Transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# SVHN dataset
dataset_svhn = datasets.SVHN('data_svhn', download=True, split='train', transform=transform)
dataloader_svhn = torch.utils.data.DataLoader(dataset_svhn, batch_size=batch_size, shuffle=True)

# CIFAR-10 dataset
dataset_cifar10 = datasets.CIFAR10('data_cifar10', download=True, train=True, transform=transform)
dataloader_cifar10 = torch.utils.data.DataLoader(dataset_cifar10, batch_size=batch_size, shuffle=True)

# CIFAR-100 dataset
dataset_cifar100 = datasets.CIFAR100('data_cifar100', download=True, train=True, transform=transform)
dataloader_cifar100 = torch.utils.data.DataLoader(dataset_cifar100, batch_size=batch_size, shuffle=True)


100%|██████████| 182040794/182040794 [00:21<00:00, 8588613.59it/s] 


100%|██████████| 170498071/170498071 [00:01<00:00, 100177563.28it/s]


Extracting data_cifar10/cifar-10-python.tar.gz to data_cifar10


100%|██████████| 169001437/169001437 [00:01<00:00, 103380848.94it/s]


Extracting data_cifar100/cifar-100-python.tar.gz to data_cifar100


In [ ]:
!pip install einops
!pip install vit_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 683.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
import torch.cuda

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0., fff_model=False):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
                PreNorm(dim, FFF(input_width=dim, leaf_width=32, output_width=dim, depth=2, activation=nn.ReLU(), dropout=0.0, train_hardened=True, region_leak=0.0)) if fff_model
                else PreNorm(dim, SimpleFeedForward(dim, mlp_dim, dim))
            ]))
    def forward(self, x):
        total_time = 0
        for attn, ff in self.layers:
            x = attn(x) + x

            #start_time = torch.cuda.Event(enable_timing=True)
            #start_time.record()
            y = ff(x)
            #end_time = torch.cuda.Event(enable_timing=True)
            #end_time.record()
            #torch.cuda.synchronize()
            #total_time += start_time.elapsed_time(end_time)

            x = x + y

        return x,total_time

class myViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0., fff_model=False):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout, fff_model=fff_model)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )
        self.total_time = 0

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)

        x = torch.cat((cls_tokens, x), dim=1)

        x += self.pos_embedding[:, :(n + 1)]

        x = self.dropout(x)

        x, one_pass_time = self.transformer(x)

        self.total_time += one_pass_time

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)

        return self.mlp_head(x), self.total_time

In [ ]:
activation = nn.ReLU()
batch_size = 256

leaf_width = 32         # [1, 2, 4, 8]
training_width = 128     # [16, 32, 64, 128]    # number of all neurons in leafs

depth = int(math.log2(training_width/leaf_width))
print(depth)
"""
depth = []
for w in leaf_width:
    for ℓ in training_width:
        d = math.log2(w / ℓ)
        depth.append(d)
"""
entropy_effect = 0.0

leaf_dropout = 0.0
region_leak = 0.0


import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from vit_pytorch.vit import ViT
import torch.optim.lr_scheduler as lr_scheduler

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 dataset with augmentations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomApply([transforms.RandomAffine(degrees=45, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.5),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)


# Model configuration
model = myViT(
    image_size=32,
    patch_size=4,
    num_classes=10,
    dim=128,
    depth=4,
    heads=8,
    mlp_dim=128,
    dim_head=64,
    dropout=0,
    emb_dropout=0.1,
    fff_model=True,
).to(device)

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=4e-4)
criterion = torch.nn.CrossEntropyLoss()

milestones = [150, 300]
gamma = 0.5
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)


# Training loop
num_epochs = 450

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs, _ = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        scheduler.step()
print("Training completed.")


# Evaluation on test set
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs,_ = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# h=0 acc=64.74 100 epochs
# h=5 acc=54.21 50 epochs
# h=10 acc=58.35 50 epochs

2
Files already downloaded and verified
Files already downloaded and verified
Training completed.
Test Accuracy: 63.56%


In [ ]:
# speedup
# only here time has meaning

activation = nn.ReLU()
batch_size = 256

leaf_width = 32         # [1, 2, 4, 8]
training_width = 128     # [16, 32, 64, 128]    # number of all neurons in leafs

depth = int(math.log2(training_width/leaf_width))
print(depth)
"""
depth = []
for w in leaf_width:
    for ℓ in training_width:
        d = math.log2(w / ℓ)
        depth.append(d)
"""
entropy_effect = 0.0

leaf_dropout = 0.0
region_leak = 0.0


import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from vit_pytorch.vit import ViT
import torch.optim.lr_scheduler as lr_scheduler

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 dataset with augmentations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomApply([transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.5),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)


# Model configuration
model = myViT(
    image_size=32,
    patch_size=4,
    num_classes=10,
    dim=128,
    depth=4,
    heads=8,
    mlp_dim=128,
    dim_head=64,
    dropout=0,
    emb_dropout=0.1,
    fff_model=True,
).to(device)

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=4e-4)
criterion = torch.nn.CrossEntropyLoss()

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=30, verbose=True)

# Evaluation loop
num_epochs = 100

total_time = 0
for epoch in range(num_epochs):

  model.eval()
  all_preds, all_labels = [], []

  with torch.no_grad():
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)

          outputs, time = model(images)
          total_time += time
          _, preds = torch.max(outputs, 1)
          all_preds.extend(preds.cpu().numpy())
          all_labels.extend(labels.cpu().numpy())

# Calculate the total number of passes
total_passes = len(test_loader.dataset)

# Calculate and print the average inference time per pass
average_time_per_batch = total_time / total_passes
print(f"Total Average Inference Time: {average_time_per_batch:.5f} milliseconds")

# Calculate overall accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# h=0 acc=64.74 100 epochs
# h=5 acc=54.21 50 epochs
# h=10 acc=58.35 50 epochs

2
Files already downloaded and verified
Files already downloaded and verified
Total Average Inference Time: 0.00000 milliseconds
Test Accuracy: 11.06%


In [ ]:
print(f"Total Average Inference Time: {average_time_per_batch*total_passes:.5f} milliseconds")

Total Average Inference Time: 444454.71875 milliseconds
